In [76]:
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [77]:
data_dir = ('./data/')
class_names = ["apple", "bat", "circle", "clock", "cloud",
               "crown", "diamond", "donut", "fish",
                "hot_dog", "lightning", "mountain", "skull",
                "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

len(class_names)

19

In [78]:
X = []
y = []

In [79]:
for label, class_name in enumerate(class_names):
    # Create the path to the .npy file (e.g., './data/apple.npy')
    file_path = os.path.join(data_dir, f"{class_name}.npy")

    # Load the .npy file → shape: (2000, 28, 28)
    data = np.load(file_path)

    X.append(data)

    # Add labels to y → [label, label, label, ..., label] (length = number of samples)
    y.append(np.full((data.shape[0]), label))


In [80]:
# Stack all image arrays vertically → final shape: (total_samples, 28, 28)
X = np.vstack(X)

# Stack all label arrays horizontally → final shape: (total_samples,)
y = np.hstack(y)

In [81]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Max Normalization

In [82]:
# # Normalize pixel values to range 0–1 (from 0–255)
# X = X.astype('float32') / 255.0
# X.shape

In [83]:
# Flatten each image from 28x28 → 784 for dense layers
# X = X.reshape(X.shape[0], -1)  # shape becomes (total_samples, 784)

In [84]:
X_train, X_, y_train, y_ = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
# `stratify=y` ensures equal class distribution in both training and validation sets.

X_cv, X_test, y_cv, y_test = train_test_split(X_,y_, test_size=0.5, random_state=42, stratify=y_)

In [85]:
model = Sequential([
    Dense(784, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(784,)),
    Dropout(0.4),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.35),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.25),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.15),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.1),
    Dense(19, activation='linear')
])

c:\Users\mehim\anaconda3\envs\ml_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [86]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [87]:
# Train
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [88]:
model.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=100, batch_size=20, callbacks=callbacks)

Epoch 1/100
22800/22800 ━━━━━━━━━━━━━━━━━━━━ 306s 13ms/step - accuracy: 0.6029 - loss: 2.2552 - val_accuracy: 0.7704 - val_loss: 1.5152 - learning_rate: 0.0010
Epoch 2/100
22800/22800 ━━━━━━━━━━━━━━━━━━━━ 313s 14ms/step - accuracy: 0.7197 - loss: 1.7094 - val_accuracy: 0.7811 - val_loss: 1.4437 - learning_rate: 0.0010
Epoch 3/100
22800/22800 ━━━━━━━━━━━━━━━━━━━━ 276s 12ms/step - accuracy: 0.7282 - loss: 1.6427 - val_accuracy: 0.7881 - val_loss: 1.3788 - learning_rate: 0.0010
Epoch 4/100
22800/22800 ━━━━━━━━━━━━━━━━━━━━ 279s 12ms/step - accuracy: 0.7290 - loss: 1.6247 - val_accuracy: 0.7929 - val_loss: 1.3607 - learning_rate: 0.0010
Epoch 5/100
22800/22800 ━━━━━━━━━━━━━━━━━━━━ 281s 12ms/step - accuracy: 0.7314 - loss: 1.6060 - val_accuracy: 0.7934 - val_loss: 1.3352 - learning_rate: 0.0010
Epoch 6/100
22800/22800 ━━━━━━━━━━━━━━━━━━━━ 280s 12ms/step - accuracy: 0.7315 - loss: 1.5870 - val_accuracy: 0.7926 - val_loss: 1.3546 - learning_rate: 0.0010
Epoch 7/100
22800/22800 ━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
# Predict logits (raw scores)
logits = model.predict(X_cv)  # shape: (num_samples, num_classes)

# Convert logits to probabilities using softmax
y_cv_pred_probs = tf.nn.softmax(logits, axis=1).numpy()

# Show probability distribution for sample 50
print("Predicted probabilities for sample 50:", y_cv_pred_probs[50])

# Get predicted class index with the highest probability
y_cv_pred = np.argmax(y_cv_pred_probs, axis=1)

# Show predicted class index for sample 50
print("Predicted class index for sample 50:", y_cv_pred[50])

# Get class label
print("Predicted class label for sample 50:", class_names[y_cv_pred[50]])
print("Actual class:", class_names[y_cv[50]])

1782/1782 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
Predicted probabilities for sample 50: [8.4675412e-05 9.7491121e-01 5.4676784e-04 2.6343580e-05 3.5018532e-03
 2.6689239e-03 1.8161867e-03 2.6996902e-04 6.8446142e-03 8.6357468e-04
 9.7350118e-04 1.0842360e-03 3.4031604e-04 4.7382171e-04 2.6246271e-04
 3.6538150e-03 2.8337256e-04 7.2878646e-04 6.6545419e-04]
Predicted class index for sample 50: 1
Predicted class label for sample 50: bat


In [ ]:
print(accuracy_score(y_cv,y_cv_pred))

0.8882631578947369


In [ ]:
logits = model.predict(X_test)
y_test_pred_probs = tf.nn.softmax(logits, axis=1).numpy() 
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
print(accuracy_score(y_test,y_test_pred))

1782/1782 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
0.8891754385964912


In [ ]:
model.save("doodle_model.keras")

In [ ]:
import joblib

data_scaler = {"scaler" : scaler}

joblib.dump(data_scaler, "data_scaler.pkl")

['data_scaler.pkl']